In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
sch=StructType().add("Id",IntegerType(),True)\
    .add("Name",StringType(),True)\
    .add("Age",StringType(),True)

    

In [0]:
df=spark.read.csv("/FileStore/Employee.csv").option("header",True).option("mode","permisive").schema(sch)

---------------------------------------------------------------------------
PySparkAttributeError                     Traceback (most recent call last)
File <command-3791464773480657>, line 1
----> 1 df=spark.read.csv("/FileStore/Employee.csv").option("header",True).option("mode","permisive").schema(sch)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3732, in DataFrame.__getattr__(self, name)
   3699 """Returns the :class:`Column` denoted by ``name``.
   3700 
   3701 .. versionadded:: 1.3.0
   (...)
   3729 +---+
   3730 """
   3731 if name not in self.columns:
-> 3732     raise PySparkAttributeError(
   3733 

In [0]:
dbutils.fs.mount(source = "wasbs://input1@adlsgmde.blob.core.windows.net",
                 mount_point ="/mnt/mountScope",
                 extra_configs = {"fs.azure.account.key.adlsgmde.blob.core.windows.net":dbutils.secrets.get(scope = "mountScope", key = "mountsecrete")})

True

In [0]:
dbutils.fs.ls("/mnt/mountScope")

[FileInfo(path='dbfs:/mnt/mountScope/sales_asisa_24_07_2024.csv', name='sales_asisa_24_07_2024.csv', size=218, modificationTime=1722594623000),
 FileInfo(path='dbfs:/mnt/mountScope/sales_emea_23_07_2024.csv', name='sales_emea_23_07_2024.csv', size=218, modificationTime=1722594623000),
 FileInfo(path='dbfs:/mnt/mountScope/sales_uk_23_07_2024.csv', name='sales_uk_23_07_2024.csv', size=214, modificationTime=1722594623000),
 FileInfo(path='dbfs:/mnt/mountScope/sales_usa_23_07_2024.csv', name='sales_usa_23_07_2024.csv', size=213, modificationTime=1722594623000),
 FileInfo(path='dbfs:/mnt/mountScope/transaction_asia_23_07_2024.csv', name='transaction_asia_23_07_2024.csv', size=185, modificationTime=1722594623000),
 FileInfo(path='dbfs:/mnt/mountScope/transaction_emea_26_07_2024.csv', name='transaction_emea_26_07_2024.csv', size=185, modificationTime=1722594624000),
 FileInfo(path='dbfs:/mnt/mountScope/transaction_uk_24_07_2024.csv', name='transaction_uk_24_07_2024.csv', size=181, modificatio

In [0]:
df=spark.read.option("header",True).option("inferschema",True).csv("/mnt/mountScope/sales_asisa_24_07_2024.csv")

In [0]:
df.display()

SaleID,SalesRepID,ProductID,Quantity,SaleDate,SaleAmount
16,116,1016,4,2023-07-01,79.96
17,117,1017,1,2023-07-01,19.99
18,118,1018,2,2023-07-01,39.98
19,119,1019,3,2023-07-01,59.97
20,120,1020,1,2023-07-01,19.99


In [0]:
agr=df.groupby("SaleID").max("SaleAmount") 

In [0]:
agr.display()

SaleID,max(SaleAmount)
16,79.96
20,19.99
19,59.97
17,19.99
18,39.98


In [0]:
#for writing in adls as csv file
df.write.csv("/mnt/mountScope/csvoutput")

In [0]:
df.write.parquet("/mnt/mountScope/parquetutput")

In [0]:
df.write.format("delta").saveAsTable("AsianSales")

In [0]:
#create the external tables
df.write.format("delta").save("/mnt/mountScope/store_table")

In [0]:
%sql
create table extr_store_table
using delta
OPTIONS (path"abfss://input1@adlsgmde.dfs.core.windows.net/store_table")

In [0]:
%sql
select * from extr_store_table

SaleID,SalesRepID,ProductID,Quantity,SaleDate,SaleAmount
16,116,1016,4,2023-07-01,79.96
17,117,1017,1,2023-07-01,19.99
18,118,1018,2,2023-07-01,39.98
19,119,1019,3,2023-07-01,59.97
20,120,1020,1,2023-07-01,19.99


In [0]:
%sql
select * from AsianSales

SaleID,SalesRepID,ProductID,Quantity,SaleDate,SaleAmount
16,116,1016,4,2023-07-01,79.96
17,117,1017,1,2023-07-01,19.99
18,118,1018,2,2023-07-01,39.98
19,119,1019,3,2023-07-01,59.97
20,120,1020,1,2023-07-01,19.99


In [0]:
%sql
describe extr_store_table

col_name,data_type,comment
SaleID,int,null
SalesRepID,int,null
ProductID,int,null
Quantity,int,null
SaleDate,date,null
SaleAmount,double,null


In [0]:
%sql
describe extended extr_store_table

col_name,data_type,comment
SaleID,int,null
SalesRepID,int,null
ProductID,int,null
Quantity,int,null
SaleDate,date,null
SaleAmount,double,null
,,
# Delta Statistics Columns,,
Column Names,"Quantity, SaleDate, SalesRepID, SaleID, ProductID, SaleAmount",
Column Selection Method,first-32,


In [0]:
%sql
delete from  extr_store_table where SaleID=16

num_affected_rows
1


In [0]:
%sql
select * from extr_store_table

SaleID,SalesRepID,ProductID,Quantity,SaleDate,SaleAmount
17,117,1017,1,2023-07-01,19.99
18,118,1018,2,2023-07-01,39.98
19,119,1019,3,2023-07-01,59.97
20,120,1020,1,2023-07-01,19.99


In [0]:
%sql
describe history   extr_store_table

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2024-08-06T10:08:22Z,1897526247183007,jhegde020@gmail.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> true)",null,List(3791464773480655),0806-045752-8s33avik,1,SnapshotIsolation,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1550, p25FileSize -> 1522, numDeletionVectorsRemoved -> 1, minFileSize -> 1522, numAddedFiles -> 1, maxFileSize -> 1522, p75FileSize -> 1522, p50FileSize -> 1522, numAddedBytes -> 1522)",null,Databricks-Runtime/14.3.x-photon-scala2.12
1,2024-08-06T10:08:11Z,1897526247183007,jhegde020@gmail.com,DELETE,"Map(predicate -> [""(SaleID#1016 = 16)""])",null,List(3791464773480655),0806-045752-8s33avik,0,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 7963, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 6215, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1728)",null,Databricks-Runtime/14.3.x-photon-scala2.12
0,2024-08-06T07:51:47Z,1897526247183007,jhegde020@gmail.com,WRITE,"Map(mode -> ErrorIfExists, statsOnLoad -> false, partitionBy -> [])",null,List(3791464773480655),0806-045752-8s33avik,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 5, numOutputBytes -> 1550)",null,Databricks-Runtime/14.3.x-photon-scala2.12


# optimisation

In [0]:
%sql
-- Optimize the table with Z-order on SaleDate
OPTIMIZE extr_store_table ZORDER BY (SaleDate);


path,metrics
abfss://input1@adlsgmde.dfs.core.windows.net/store_table,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1522), 0, List(0, 0), 0, null), 0, 1, 1, false, 0, 0, 1722940866954, 1722940867262, 4, 0, null, List(0, 0), 6, 6, 0, 0, null)"


In [0]:
%sql
select * from extr_store_table

SaleID,SalesRepID,ProductID,Quantity,SaleDate,SaleAmount
17,117,1017,1,2023-07-01,19.99
18,118,1018,2,2023-07-01,39.98
19,119,1019,3,2023-07-01,59.97
20,120,1020,1,2023-07-01,19.99
18,121,1011,2,2023-07-01,34.2


In [0]:
%sql
insert into extr_store_table values(25,121,1011,2,'2023-07-01',34.2)

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
-- compaction

optimize extr_store_table

path,metrics
abfss://input1@adlsgmde.dfs.core.windows.net/store_table,"List(1, 4, List(1644, 1644, 1644.0, 1, 1644), List(1368, 1522, 1406.5, 4, 5626), 0, null, 1, 4, 0, true, 0, 0, 1722944238536, 1722944248704, 4, 1, null, List(0, 0), 6, 6, 2155, 0, null)"


In [0]:
%sql
describe history   extr_store_table

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2024-08-06T11:34:57Z,1897526247183007,jhegde020@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3791464773480655),0806-045752-8s33avik,4,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1368)",null,Databricks-Runtime/14.3.x-photon-scala2.12
4,2024-08-06T11:34:43Z,1897526247183007,jhegde020@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3791464773480655),0806-045752-8s33avik,3,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1368)",null,Databricks-Runtime/14.3.x-photon-scala2.12
3,2024-08-06T11:27:16Z,1897526247183007,jhegde020@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3791464773480655),0806-045752-8s33avik,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1368)",null,Databricks-Runtime/14.3.x-photon-scala2.12
2,2024-08-06T10:08:22Z,1897526247183007,jhegde020@gmail.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> true)",null,List(3791464773480655),0806-045752-8s33avik,1,SnapshotIsolation,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1550, p25FileSize -> 1522, numDeletionVectorsRemoved -> 1, minFileSize -> 1522, numAddedFiles -> 1, maxFileSize -> 1522, p75FileSize -> 1522, p50FileSize -> 1522, numAddedBytes -> 1522)",null,Databricks-Runtime/14.3.x-photon-scala2.12
1,2024-08-06T10:08:11Z,1897526247183007,jhegde020@gmail.com,DELETE,"Map(predicate -> [""(SaleID#1016 = 16)""])",null,List(3791464773480655),0806-045752-8s33avik,0,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 7963, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 6215, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1728)",null,Databricks-Runtime/14.3.x-photon-scala2.12
0,2024-08-06T07:51:47Z,1897526247183007,jhegde020@gmail.com,WRITE,"Map(mode -> ErrorIfExists, statsOnLoad -> false, partitionBy -> [])",null,List(3791464773480655),0806-045752-8s33avik,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 5, numOutputBytes -> 1550)",null,Databricks-Runtime/14.3.x-photon-scala2.12


In [0]:
%sql
-- Optimize the table with Z-order on SaleDate
OPTIMIZE extr_store_table ZORDER BY (SaleDate);

path,metrics
abfss://input1@adlsgmde.dfs.core.windows.net/store_table,"List(1, 4, List(1660, 1660, 1660.0, 1, 1660), List(1368, 1644, 1437.0, 4, 5748), 0, List(minCubeSize(107374182400), List(0, 0), List(4, 5748), 0, List(4, 5748), 1, null), 1, 4, 0, false, 0, 0, 1722944573448, 1722944580711, 4, 1, null, List(0, 0), 6, 6, 1340, 0, null)"


# Temporary tables

In [0]:
df.display()

SaleID,SalesRepID,ProductID,Quantity,SaleDate,SaleAmount
16,116,1016,4,2023-07-01,79.96
17,117,1017,1,2023-07-01,19.99
18,118,1018,2,2023-07-01,39.98
19,119,1019,3,2023-07-01,59.97
20,120,1020,1,2023-07-01,19.99


In [0]:
df.createOrReplaceTempView("tempSales")

In [0]:
%sql
select * from tempSales

SaleID,SalesRepID,ProductID,Quantity,SaleDate,SaleAmount
16,116,1016,4,2023-07-01,79.96
17,117,1017,1,2023-07-01,19.99
18,118,1018,2,2023-07-01,39.98
19,119,1019,3,2023-07-01,59.97
20,120,1020,1,2023-07-01,19.99
